In [61]:
import pandas as pd
import numpy as np
import time
import collections

# 2,3,4 mm
#delay_lst = ['10ns','20ns','30ns','40ns','50ns','100ns','200ns','350ns','500ns','750ns','1000ns','2000ns','3500ns','5000ns','7500ns','10000ns','20000ns']
#delay_lst = ['10ns','20ns','30ns','40ns','50ns','100ns','200ns','500ns','1000ns','2000ns','5000ns','10000ns']

# for pure helium case
#delay_lst = ['0ns','10ns','20ns','30ns','40ns','50ns','100ns','200ns','350ns','500ns','750ns','1000ns','2000ns','3500ns','5000ns','7500ns','10000ns']

# 5,6 mm
delay_lst = [f"{i}ns" for i in range(-2, 32, 2)]
lst2 = ["35ns", "40ns", "45ns", "50ns"]
delay_lst.extend(lst2)
print(delay_lst)
# input "Air" to include ambient air data
height_lst = ['Air']

file_dir = '/Users/YK/Desktop/'

folder = '24_06_27_water_air_early_cam1'

# #water heights table for Danny's data
# height_tab = 'water_height_in_pixel'

#length of the strip in um
strip_length = 250

# focal plane location
pix_focal_plane = 589 #assuming the focal plane is 10 pixels lower than the new camera(531)

# setup input "D" for Danny's setup, "C" for current setup
setup = "C"

# number of runs per delay
count_runs = 5

# upper bound of the detection efficiency at 970, lower at 190
# if no detection efficiency, change upper bound to 1024, lower bound to 0
low_bound = 190
upp_bound = 970

    
#read calibration data
detect_eff = pd.read_csv('/Users/YK/Desktop/calibration/box_DE.csv')
#detect_eff = pd.DataFrame(np.ones((1024,1024)))
wavelen_calib = pd.read_csv('/Users/YK/Desktop/calibration/wavelen_calib.csv')
# collect_eff = pd.read_csv('/Users/YK/Desktop/calibration/collect_eff.csv',header = None)

if setup == 'D':
    Pixel_Size = pd.read_csv('/Users/YK/Desktop/calibration/True_Pixel_Size.csv')
else:
    Pixel_Size = pd.read_csv('/Users/YK/Desktop/calibration/True_Pixel_Size_box.csv')
    
print("ok")

['-2ns', '0ns', '2ns', '4ns', '6ns', '8ns', '10ns', '12ns', '14ns', '16ns', '18ns', '20ns', '22ns', '24ns', '26ns', '28ns', '30ns', '35ns', '40ns', '45ns', '50ns']
ok


In [62]:
#match each element in delay_list with its corresponding gain
Mydict = dict()

def get_int(s):
    num = ''
    for i in s:
        if i != 'n':
            num += i
        else:
            break
    return int(num)

def gain_select_1_21_24(delay_lst,height):
    if height not in ['5mm','6mm']:
        for s in delay_lst:
            delay = get_int(s)
            if delay <= 200:
                Mydict[s] = '200g_4ttl'
            elif delay <= 2000:
                Mydict[s] = '200g_4ttl'
            elif height == '4mm' and delay == '20000ns':
                Mydict[s] = '200g_4ttl'
            else:
                Mydict[s] = '200g_4ttl'
    else:
        for s in delay_lst:
            delay = get_int(s)
            if delay <= 200:
                Mydict[s] = '200g_4ttl'
            elif delay <= 2000:
                Mydict[s] = '200g_4ttl'
            else:
                Mydict[s] = '200g_4ttl'
    return Mydict

def gain_select_2_24_24(delay_lst,height):
    if height == '0mm':
        for s in delay_lst:
            delay = get_int(s)
            if delay >= 10 and delay <= 40:
                Mydict[s] = '100g_5ttl'
            elif delay <= 100 or delay <= 0:
                Mydict[s] = '200g_5ttl'
            elif delay < 10000:
                Mydict[s] = '250g_5ttl'
            elif delay == 10000:
                Mydict[s] = '250g_50ttl'
            else:
                Mydict[s] = '250g_500ttl'
    elif height == '1mm':
        for s in delay_lst:
            delay = get_int(s)
            if delay <= 100:
                Mydict[s] = '200g_5ttl'
            elif delay < 5000:
                Mydict[s] = '250g_5ttl'
            elif delay < 20000:
                Mydict[s] = '250g_50ttl'
            else:
                Mydict[s] = '250g_500ttl'
    elif height == '2mm':
        for s in delay_lst:
            delay = get_int(s)
            if delay <= 100:
                Mydict[s] = '200g_5ttl'
            elif delay < 5000:
                Mydict[s] = '250g_5ttl'
            elif delay < 10000:
                Mydict[s] = '250g_50ttl'
            else:
                Mydict[s] = '250g_500ttl'
    else:
        for s in delay_lst:
            delay = get_int(s)
            if delay <= 200:
                Mydict[s] = '200g_5ttl'
            elif delay <= 2000:
                Mydict[s] = '250g_5ttl'
            else:
                Mydict[s] = '250g_50ttl'
    return Mydict


def intensity_corr_1_21_24(delay,height):
    # correction for gains and gate width for intensity comparison across various delays 
    int_delay = int(delay[:-2])

    if height in ['Air','5mm','6mm']:
        if int_delay > 100:
            return 0.311923 #bg 250g 5ttl   #bg 250g 50 ttl 0.0311923
        else:
            return 0.8 # 200g 4ttl  1 for 200g 5 ttl
    else:
        if int_delay <= 100:
            return 0.8
        elif height == '4mm' and int_delay == 20000:
            return 0.00311923
        elif int_delay >= 200 and int_delay < 5000:
            return 0.311923
        elif int_delay >= 5000:
            return 0.0311923
        
def intensity_corr_2_24_24(delay,height):
    # correction for gains and gate width for intensity comparison across various delays 
    int_delay = int(delay[:-2])

    if height in ['Air','5mm','6mm']:
        if int_delay > 100:
            return 0.311923 #bg 250g 5ttl
        else:
            return 1 # 200g 5ttl
    elif height == '0mm':
        if int_delay >= 10 and int_delay <= 40:
            return 10.2779
        elif int_delay <= 100 or int_delay <= 0:
            return 1
        elif int_delay < 10000:
            return 0.311923
        elif int_delay == 10000:
            return 0.0311923
        else:
            return 0.00311923
    elif height == '1mm':
        if int_delay <= 100:
            return 1
        elif int_delay < 5000:
            return 0.311923
        elif int_delay < 20000:
            return 0.0311923
        else:
            return 0.00311923
    elif height == '2mm':
        if int_delay <= 100:
            return 1
        elif int_delay < 5000:
            return 0.311923
        elif int_delay < 10000:
            return 0.0311923
        else:
            return 0.00311923

In [63]:
if folder == '24_06_27_water_air_early_cam1':
    intensity_corr = intensity_corr_1_21_24
    gain_select = gain_select_1_21_24
else:
    intensity_corr = intensity_corr_2_24_24
    gain_select = gain_select_2_24_24

In [64]:
def cal_water_height(pix_focal_plane,target_height):
    # calculate water surface pixel given the focal plane pixel and the height in the stepping motor
    pixel_size = pd.read_csv('/Users/YK/Desktop/calibration/True_Pixel_Size_box.csv')
    idx_focal = pixel_size.loc[pixel_size['Pixel']==pix_focal_plane].index[0]

    height_um = target_height*10**3
    count_um = 0
    idx_tracker = idx_focal-1
    while count_um < height_um:
        count_um += pixel_size.iloc[idx_tracker,2]
        idx_tracker -= 1
    return idx_tracker

def cal_dist(pix_focal_plane,target_pix):
    # calculate the distance in mm between a target pixel position and the focal plane
    pixel_size = pd.read_csv('/Users/YK/Desktop/calibration/True_Pixel_Size_box.csv')
    
    idx_focal = pixel_size.loc[pixel_size['Pixel']==pix_focal_plane].index[0]
    count_um = 0
    
    if target_pix < pix_focal_plane:
        idx_tracker = idx_focal-1
        while idx_tracker >= target_pix:
            count_um += pixel_size.iloc[idx_tracker,2]
            idx_tracker -= 1
        return -count_um/1000
    else:
        idx_tracker = idx_focal
        while idx_tracker < target_pix:
            count_um += pixel_size.iloc[idx_tracker,2]
            idx_tracker += 1
        return count_um/1000

def spatial_axis(pix_focal_plane,Pixel_Size):
    # change the spatial axis from pixel to mm based on the magnification
    spat_tab = pd.DataFrame()
    spat_tab['pix'] = np.linspace(1,1024,num=1024)
    
    # z [mm] for first pix
    z = cal_dist(pix_focal_plane,0)
    z_lst = [z]
    for idx in range(0,1023):
        z += Pixel_Size.iloc[idx,2]/1000
        z_lst.append(z)
    spat_tab['z'] = z_lst
    return spat_tab
    
def taking_strip(strip_length,surface,upp_bound,Pixel_Size):
    s = surface
    count = 0
    lst_range = []
    start = s
    while s < upp_bound:
        count += Pixel_Size.iloc[s,2]
        if count >= strip_length:
            end = s
            lst_range.append(range(start-surface,end-surface+1))
            count = 0
            start = end+1
        s += 1
    return lst_range

def pre_process(df,bg,detect_eff,surface,upp_bound):
    # input the raw dataframe, background, detection efficiency, and surface height
    # does background subtraction, detection efficiency correction, and cut-off at the water surface
    df -= bg
    df = df.drop(columns = df.columns[0]).drop(columns = df.columns[1025])
    a = df.to_numpy()
    b = detect_eff.to_numpy()
    a = a.reshape(-1,1)
    b = b.reshape(-1,1)
    c = (a/b).reshape(1024,1024)
    #transpose the matrix such that it corresponds to the image in andor camera
    df = pd.DataFrame(c).T
        
    #cut off the dataframe at the waterheight
    df = df.iloc[surface:upp_bound,:]
        
    #if height_lst[i] != 'Air':
        #collect_eff = collect_eff.iloc[:1024-surface,:]
        #a1 = df.to_numpy()
        #b1 = collect_eff.to_numpy()
        #a1 = a1.reshape(-1,1)
        #b1 = b1.reshape(-1,1)
        #c1 = (a1/b1).reshape(1024-surface,1024)
        #df = pd.DataFrame(c1)
    return df

def SR_spec(df,lst_range,lst_header):
    # generate spatially resolved spectrum
    # input the processed dataframe and the range of the strips
    spec = pd.DataFrame() 
    spec['wavelen'] = wavelen_calib['wavelen']
    spec['pix'] = wavelen_calib['pix']

    for strip in lst_range:
        spec[str(strip)] = np.sum(df.iloc[strip,:],axis=0)
    spec.columns = [['wavelen','pix']+lst_header]
    return spec

def SI_spec(df):
    # generate spatially integrated spectrum
    # input dataframe, low and upper bound for vertical binning
    spec = pd.DataFrame()
    spec['wavelen'] = wavelen_calib['wavelen']
    spec['pix'] = wavelen_calib['pix']
    spec['sig'] = np.sum(df,axis=0)
    return spec

In [55]:

cal_water_height(pix_focal_plane,5)



106

In [65]:
#binning for spatial and spectral analysis
t0 = time.time()
spat_tab = spatial_axis(pix_focal_plane,Pixel_Size)

# go through all the water heights
for i in range(len(height_lst)):
    print(height_lst[i])
    # select or calculate the water surface position
    if setup == 'D':
        #select water surface using the heights table
        tab = pd.read_csv(file_dir+'Water_Heights/'+height_tab+'.csv')
        #the index slicing has to be changed when using a different folder name
        take = folder
        if take in tab['take'].values and height_lst[i] in tab.columns:
            surface = tab.loc[tab['take'] == take,[height_lst[i]]].values[0][0]
        else:
            surface = 30
    else:
        if height_lst[i] != 'Air':
            surface_pix = cal_water_height(pix_focal_plane,int(height_lst[i][0]))
            if surface_pix < low_bound:
                surface = low_bound
                air_height = cal_dist(pix_focal_plane,low_bound)
            else:
                surface = surface_pix
        else:
            surface = low_bound
            air_height = cal_dist(pix_focal_plane,surface)
    
    # take strips
    lst_range = taking_strip(strip_length,surface,upp_bound,Pixel_Size)
    
    # convert header from pixel ranges to mm
    tracker = 0.125
    lst_header = []
    for _ in range(len(lst_range)):
        if height_lst[i] != 'Air' and surface != low_bound:
            height = -int(height_lst[i][0])
        else:
            height = air_height
        lst_header.append(str(tracker+height))
        tracker += 0.25
    
    spat_dist = pd.DataFrame() 
    spat_dist['distance'] = lst_header
    
    # go through all the delays
    for j in range(len(delay_lst)):
        
        #read background
        Mydict = gain_select(delay_lst,height_lst[i])
        gain = Mydict[delay_lst[j]]
        bg_file = file_dir+folder+'/'+height_lst[i]+'/ASCII/'+'bg_'+gain+'.asc'
        bg = pd.read_csv(bg_file, delimiter = '\t', header = None)
        
        spat_lst = [] # for spatial distribution with 250um binned strips 
        spat_pix = pd.DataFrame() # for spatial distribution with respect to 1024 pixels
        
        spat_pix['z'] = spat_tab['z'].iloc[surface:upp_bound]

        # go through indiviual runs
        for r in range(count_runs):
            run_num = r+1
            file = file_dir+folder+'/'+height_lst[i]+'/ASCII/'+delay_lst[j]+'_'+str(run_num)+'.asc'
            df_raw = pd.read_csv(file, delimiter = '\t', header = None)
            df = pre_process(df_raw,bg,detect_eff,surface,upp_bound)*intensity_corr(delay_lst[j],height_lst[i])
            if r == 0:
                df_averaged = df/count_runs
            else:
                df_averaged += df/count_runs
            
            # spatially resolved spectrum
            SR_spectrum = SR_spec(df,lst_range,lst_header)
            
            # spatial distribution with 250 um strip
            spat_arr = (np.sum(SR_spectrum.iloc[:,2:],axis=0)-np.sum(SR_spectrum.iloc[340:550,2:],axis=0)).to_numpy()
            spat_lst.append(spat_arr)
            
            # spatial distribution with respect to pixels
            spat_pix[str(r+1)] = (np.sum(df,axis=1)-np.sum(df.iloc[:,340:550],axis=1)).to_numpy()
        
        # averaged spatial distribution and uncertainty
        spat = np.array(spat_lst)
        avg_spat = np.mean(spat,axis = 0)
        std_spat = np.std(spat,axis = 0)
        spat_dist[delay_lst[j]] = avg_spat
        spat_dist[delay_lst[j]+'_std'] = std_spat
        
#         # averaged spatially resolved spectrum
#         SR_spec_avg = SR_spec(df_averaged,lst_range,lst_header)
        
#         # averaged spatially integrated spectrum
#         SI_spec_avg = SI_spec(df_averaged)
        
#         # data export of spectra
#         SR_spec_avg.to_csv(file_dir+'data/'+folder+'/'+height_lst[i]+'/'+delay_lst[j]+'_SR_250um.csv',index = None)
        
#         SI_spec_avg.to_csv(file_dir+'data/'+folder+'/'+height_lst[i]+'/'+delay_lst[j]+'_SI.csv',index = None)
        
        # data export of individual spatial distribution for COI analysis
        spat_pix.to_csv(file_dir +folder+'/'+height_lst[i]+'/CSV/'+delay_lst[j]+'_spatial.csv',index = None)
    
    # data export of spatial distribution
    spat_dist.to_csv(file_dir+folder+'/'+height_lst[i]+'_Spatial.csv',index = None)
        
t1 = time.time()
print((t1-t0))

Air
8.57871413230896
